In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [2]:
import mlflow
from mlflow.models import infer_signature

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/examples/sklearn_elasticnet_wine/wine-quality.csv")
data["quality"].unique()

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)

In [4]:
X = data.drop('quality', axis=1)
y = data['quality']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
def train_model(params, epochs, train_x=X_train_scaled, train_y=y_train, valid_x=X_test_scaled, valid_y=y_test, test_x=X_test_scaled, test_y=y_test):
    
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    
    model = keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"],
            momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )
    
    with mlflow.start_run(nested=True):
        model.fit(
            train_x, train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64
        )
        
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]
        
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)
        
        signature = infer_signature(train_x, model.predict(train_x))
        mlflow.tensorflow.log_model(model, "model", signature=signature)
        
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [7]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x=X_train_scaled,
        train_y=y_train,
        valid_x=X_test_scaled,
        valid_y=y_test,
        test_x=X_test_scaled,
        test_y=y_test,
    )
    return result

In [8]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [9]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    signature = infer_signature(X_train_scaled, best_run["model"].predict(X_train_scaled))
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


Epoch 1/3                                            

 1/54 ━━━━━━━━━━━━━━━━━━━━ 28s 532ms/step - loss: 31.0535 - root_mean_squared_error: 5.5726
45/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29.0197 - root_mean_squared_error: 5.3818   
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 27.9034 - root_mean_squared_error: 5.2729 - val_loss: 9.1655 - val_root_mean_squared_error: 3.0275

Epoch 2/3                                            

 1/54 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 8.7577 - root_mean_squared_error: 2.9593
50/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.0731 - root_mean_squared_error: 2.4493 
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.9119 - root_mean_squared_error: 2.4152 - val_loss: 2.8947 - val_root_mean_squared_error: 1.7014

Epoch 3/3                                            

 1/54 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 2.0696 - root_mean_squared_error: 1.4386
53/54 ━━━━━━━━━━━━━━━━━━━━ 0s 962us/step - loss: 2.7071 - root_mean_squared_error: 1.6436
5